In [1]:
import inspect

# Basics

In [61]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    BaseMessage,
    HumanMessage, 
    AIMessage, 
    SystemMessage 
)

## Message Types

In [70]:
# All message types inherit from this class
# Each message consists of content (the message itself), a type (who sent the message), 
# and additional kwargs (which are specific to the API provider)
print(inspect.getsource(BaseMessage))

class BaseMessage(BaseModel):
    """Message object."""

    content: str
    additional_kwargs: dict = Field(default_factory=dict)

    @property
    @abstractmethod
    def type(self) -> str:
        """Type of the message, used for serialization."""



In [71]:
# SystemMessage is a message containing system instructions
print(inspect.getsource(SystemMessage))

class SystemMessage(BaseMessage):
    """Type of message that is a system message."""

    @property
    def type(self) -> str:
        """Type of the message, used for serialization."""
        return "system"



In [72]:
# HumanMessage is a message sent by the user
# For few shot examples, we can set the additional attribute `example` to True
print(inspect.getsource(HumanMessage))

class HumanMessage(BaseMessage):
    """Type of message that is spoken by the human."""

    example: bool = False

    @property
    def type(self) -> str:
        """Type of the message, used for serialization."""
        return "human"



In [73]:
# AIMessage is a message sent by the chatbot
# For few shot examples, we can set the additional attribute `example` to True
print(inspect.getsource(AIMessage))

class AIMessage(BaseMessage):
    """Type of message that is spoken by the AI."""

    example: bool = False

    @property
    def type(self) -> str:
        """Type of the message, used for serialization."""
        return "ai"



## Calling a Chat Model

In [74]:
chat_model = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

In [83]:
# Different from LLMs, calling a chat model requires a list of messages, not just
# a single prompt, so we wrap our message in a list
response = chat_model([
    HumanMessage(content="Hello, how are you?")
])
response

AIMessage(content="As an AI language model, I don't have feelings, but I'm functioning well. How can I assist you today?", additional_kwargs={}, example=False)

In [81]:
# Passing a list of messages
response = chat_model([
    HumanMessage(content="Hello, how are you?"),
    AIMessage(content="As an AI language model, I don't have feelings, but I'm functioning well. How can I assist you today?"),
    HumanMessage(content="Can you remind me what my initial question was about?"),
])
response

AIMessage(content='Your initial question was "Hello, how are you?"', additional_kwargs={}, example=False)

In [82]:
# Using system messages, we can add general instructions to the chat model
response = chat_model([
    SystemMessage(content="You are a chatbot. You will be small talking with a human. Please be empathic and follow usual social conventions. Don't mention that you are a chatbot."),
    HumanMessage(content="Hello, how are you?"),
])
response

AIMessage(content="Hello! I'm doing well, thank you. How about you?", additional_kwargs={}, example=False)

In [84]:
# Similarly to LLMs, we can use the `generate` method to generate multiple responses
response = chat_model.generate([
    [
        HumanMessage(content="Hello, how are you?")
    ],
    [
        SystemMessage(content="You are a chatbot. You will be small talking with a human. Please be empathic and follow usual social conventions. Don't mention that you are a chatbot."),
        HumanMessage(content="Hello, how are you?"),
    ]
])
response

LLMResult(generations=[[ChatGeneration(text="As an AI language model, I don't have feelings, but I'm functioning well. How can I assist you today?", generation_info=None, message=AIMessage(content="As an AI language model, I don't have feelings, but I'm functioning well. How can I assist you today?", additional_kwargs={}, example=False))], [ChatGeneration(text="Hello! I'm doing well, thank you. How about you?", generation_info=None, message=AIMessage(content="Hello! I'm doing well, thank you. How about you?", additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 68, 'completion_tokens': 39, 'total_tokens': 107}, 'model_name': 'gpt-3.5-turbo'})

### Annotate Few Shot Examples

In [85]:
response = chat_model([
    SystemMessage(content="You are a chatbot that reacts with emojis to input messages of a user. Please don't use words in your responses, only emojis."),
    HumanMessage(content="I like pizza", example=True),
    AIMessage(content="🍕", example=True),
    HumanMessage(content="The weather is so nice today", example=True),
    AIMessage(content="🌞", example=True),
    HumanMessage(content="I will go skiing tomorrow", example=True),
    AIMessage(content="⛷️", example=True),
    HumanMessage(content="She gave me a kiss"),
])
response

AIMessage(content='💋', additional_kwargs={}, example=False)